In [177]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
# Common imports
import numpy as np
import os
#imports pandas
import pandas as pd
from IPython.display import Image
import os
#import warnings and Repress Warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", message="numpy.ndarray size changed")
warnings.filterwarnings('ignore')
# to make this notebook's output stable across runs
np.random.seed(42)
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid") #White Grid
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import warnings
warnings.filterwarnings('ignore')

1. Download dei dati

In [178]:
#ORIGINAL DATASET https://www.kaggle.com/datasets/usmanshuaibumusa/cicids-17

df1 = pd.read_csv("dataset/archive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv");
df2 = pd.read_csv("dataset/archive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv");
df3 = pd.read_csv("dataset/archive/Friday-WorkingHours-Morning.pcap_ISCX.csv");
df4 = pd.read_csv("dataset/archive/Monday-WorkingHours.pcap_ISCX.csv");
df5 = pd.read_csv("dataset/archive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv");
#df6 = pd.read_csv("dataset/archive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv");
df7 = pd.read_csv("dataset/archive/Tuesday-WorkingHours.pcap_ISCX.csv");
df8 = pd.read_csv("dataset/archive/Wednesday-workingHours.pcap_ISCX.csv");


In [179]:
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2660377 entries, 0 to 692702
Data columns (total 85 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Flow ID                       object 
 1    Source IP                    object 
 2    Source Port                  int64  
 3    Destination IP               object 
 4    Destination Port             int64  
 5    Protocol                     int64  
 6    Timestamp                    object 
 7    Flow Duration                int64  
 8    Total Fwd Packets            int64  
 9    Total Backward Packets       int64  
 10  Total Length of Fwd Packets   float64
 11   Total Length of Bwd Packets  float64
 12   Fwd Packet Length Max        float64
 13   Fwd Packet Length Min        float64
 14   Fwd Packet Length Mean       float64
 15   Fwd Packet Length Std        float64
 16  Bwd Packet Length Max         float64
 17   Bwd Packet Length Min        float64
 18   Bwd Packet Length Mean

In [181]:
pd.DataFrame(df.isnull().sum()/df.shape[0]).sort_values(ascending=0,by=[0])

,0
Flow Bytes/s,0.000503
Flow ID,0.000000
Fwd Avg Bulk Rate,0.000000
Fwd Avg Bytes/Bulk,0.000000
Fwd Header Length.1,0.000000
...,...
Fwd IAT Mean,0.000000
Fwd IAT Total,0.000000
Flow IAT Min,0.000000
Flow IAT Max,0.000000


Elimino righe con Flow Byts/s null

In [182]:
rows = df.loc[df['Flow Bytes/s'].isnull()]
df.drop(labels=rows.index,axis=0,inplace=True)

In [183]:
df.shape

(2652969, 85)

Drop valori che sono infinito o -infinito

In [184]:
rows = df.loc[np.isinf(df.select_dtypes(exclude=object)).any(1)]
df.drop(labels=rows.index,axis=0,inplace=True)


In [185]:
df.shape

(2645598, 85)

In [186]:
df.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,...,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06,2.645598e+06
mean,4.119495e+04,8.074466e+03,9.797346e+00,1.494462e+07,9.012956e+00,9.932867e+00,5.491804e+02,1.521742e+04,2.102181e+02,1.848443e+01,...,5.015315e+00,-2.903644e+03,8.204000e+04,4.081138e+04,1.525342e+05,5.882103e+04,8.619543e+06,5.274688e+05,9.017417e+06,8.207746e+06
std,2.218952e+04,1.827980e+04,5.235527e+00,3.376377e+07,7.211856e+02,9.594596e+02,1.014595e+04,2.169947e+06,7.301311e+02,6.151993e+01,...,5.987986e+02,1.121127e+06,6.476965e+05,3.949956e+05,1.026140e+06,5.748674e+05,2.413247e+07,4.724851e+06,2.488675e+07,2.386314e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.297400e+04,5.300000e+01,6.000000e+00,1.520000e+02,2.000000e+00,1.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.094900e+04,8.000000e+01,6.000000e+00,3.132700e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.220000e+02,3.700000e+01,2.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.833400e+04,4.430000e+02,1.700000e+01,3.416831e+06,5.000000e+00,4.000000e+00,1.960000e+02,5.100000e+02,8.600000e+01,3.600000e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,6.553500e+04,1.700000e+01,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,...,2.135570e+05,1.380000e+02,1.070000e+08,7.420000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


Drop colonna Flow ID poiché info contenute già in altre colonne

In [187]:
df["Flow ID"].head()

0    192.168.10.5-104.16.207.165-54865-443-6
1      192.168.10.5-104.16.28.216-55054-80-6
2      192.168.10.5-104.16.28.216-55055-80-6
3    192.168.10.16-104.17.241.25-46236-443-6
4    192.168.10.5-104.19.196.102-54863-443-6
Name: Flow ID, dtype: object

In [188]:
df.drop(["Flow ID"," Source IP"," Destination IP"," Timestamp"],axis=1,inplace=True)


In [189]:
df[" Label"].value_counts()

BENIGN              2094126
DoS Hulk             229259
PortScan             157756
DDoS                 127156
DoS GoldenEye         10289
FTP-Patator            7901
SSH-Patator            5861
DoS slowloris          5772
DoS Slowhttptest       5489
Bot                    1944
Infiltration             34
Heartbleed               11
Name:  Label, dtype: int64

In [190]:
df_maligne = df[df[" Label"]!='BENIGN']
df_maligne.shape

(551472, 81)

In [191]:
#df_maligne[' Destination Port'].value_counts()

In [192]:
df_maligne[" Label"] = "MALIGN"
df_maligne[" Label"].value_counts()

MALIGN    551472
Name:  Label, dtype: int64

In [193]:
df_benign = df[df[" Label"]=='BENIGN'].sample(553200)
df_benign.shape

(553200, 81)

Primo test di classificazione

In [194]:
df_final = pd.concat([df_benign,df_maligne])
df_final[" Label"].value_counts()

BENIGN    553200
MALIGN    551472
Name:  Label, dtype: int64

In [195]:
from sklearn.model_selection import cross_val_score
def compute_performance(estimators, X, y, scoring_funs, cross_val=True):
    '''
    Compute every score function for each estimator
    
    Parameters:
    ----------
        estimators : a list of predictors
        X : data used for computed the corss_validation_test
        y : target values
        scoring_fun : list of performances measures
    
    Returns:
    --------
        A dictionary containing the score of each estimator
    '''
    score_dict = {}
    for e in estimators:
        score_dict[e.__class__.__name__] = {}
        if not cross_val:
            # we call predict on the estimator
            y_pred = e.predict(X)

        for f in scoring_funs: 
            if cross_val:
                score = cross_val_score(e, X ,y, cv=10 ,scoring="accuracy").mean()
                name = f
            else:
                _f, name = f
                score = _f(y_pred, y)
            score_dict[e.__class__.__name__][name] = score
    
    return score_dict

In [196]:
from sklearn.model_selection import train_test_split



y = df_final[" Label"]
X = df_final.drop(columns= " Label")

y.shape

(1104672,)

In [197]:
X.shape

(1104672, 80)

In [198]:
X_train, X_test,  y_train, y_test = train_test_split(X,y, test_size=0.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(883737, 80) (883737,) (220935, 80) (220935,)


In [199]:
y_train.value_counts()

BENIGN    442414
MALIGN    441323
Name:  Label, dtype: int64

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2645598 entries, 0 to 692702
Data columns (total 81 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Source Port                  int64  
 1    Destination Port             int64  
 2    Protocol                     int64  
 3    Flow Duration                int64  
 4    Total Fwd Packets            int64  
 5    Total Backward Packets       int64  
 6   Total Length of Fwd Packets   float64
 7    Total Length of Bwd Packets  float64
 8    Fwd Packet Length Max        float64
 9    Fwd Packet Length Min        float64
 10   Fwd Packet Length Mean       float64
 11   Fwd Packet Length Std        float64
 12  Bwd Packet Length Max         float64
 13   Bwd Packet Length Min        float64
 14   Bwd Packet Length Mean       float64
 15   Bwd Packet Length Std        float64
 16  Flow Bytes/s                  float64
 17   Flow Packets/s               float64
 18   Flow IAT Mean         

In [211]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=1, random_state=42)

rnd_clf.fit(X_train,y_train)
compute_performance([rnd_clf], X_train, y_train, scoring_funs=['accuracy', 'recall', 'precision', 'f1'])

{'RandomForestClassifier': {'accuracy': 0.998278899657676,
  'recall': 0.998278899657676,
  'precision': 0.998278899657676,
  'f1': 0.998278899657676}}

In [215]:
for i,x in enumerate(rnd_clf.feature_importances_):
    print("Position:{}: {}".format(i,x))

Position:0: 0.11563620862464351
Position:1: 0.038019318077121445
Position:2: 0.0
Position:3: 0.01918726635483687
Position:4: 0.017251661881059487
Position:5: 4.52621563607512e-06
Position:6: 0.001853004616954736
Position:7: 0.0007157558026856652
Position:8: 0.002411237800022745
Position:9: 0.002672071444310612
Position:10: 0.007210270132064745
Position:11: 0.009493533416099646
Position:12: 0.005885920604014895
Position:13: 0.0
Position:14: 0.010695273370523175
Position:15: 0.0005286021627171853
Position:16: 0.0018647729454474504
Position:17: 0.021054157586196613
Position:18: 0.0010158777352265952
Position:19: 0.0005805307564929093
Position:20: 0.0003709581821118916
Position:21: 0.0028504764276886885
Position:22: 0.00034880537676860437
Position:23: 0.0004886770526337191
Position:24: 0.15553517052444157
Position:25: 0.004727986512404113
Position:26: 0.0027247549007461307
Position:27: 0.00913265630304269
Position:28: 0.08543180954351108
Position:29: 3.09273797808232e-05
Position:30: 0.000